In [ ]:
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1"
# !pip install --upgrade -qqq uv
!pip install -qqq --upgrade unsloth vllm
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.2/438.2 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2

In [ ]:
from unsloth import FastLanguageModel
import torch
import pandas as pd
import numpy as np
from datasets import Dataset
import re

# ============================================================================
# STEP 1: Load Model
# ============================================================================
max_seq_length = 1024  # Shorter for classification tasks
lora_rank = 32

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Base",
    max_seq_length = max_seq_length,
    load_in_4bit = False,
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.9,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 10-07 14:43:47 [__init__.py:216] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 10-07 14:43:59 [vllm_utils.py:689] Unsloth: Patching vLLM v1 graph capture
Unsloth: Could not patch vLLM V0 graph capture: No module named 'vllm.worker.model_runner'
==((====))==  Unsloth 2025.10.1: Fast Qwen3 patching. Transformers: 4.55.4. vLLM: 0.11.0.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-4B-Base with actual GPU utilization = 89.09%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 2

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.10.1 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [ ]:
# ============================================================================
# STEP 2: Define Custom Tags and System Prompt
# ============================================================================
reasoning_start = "<start_working_out>"
reasoning_end   = "<end_working_out>"
solution_start  = "<SOLUTION>"
solution_end    = "</SOLUTION>"

system_prompt = f"""You are a botanist analyzing iris flowers.
Given measurements of an iris flower, analyze the features carefully.
Place your reasoning between {reasoning_start} and {reasoning_end}.
Then, provide the species name between {solution_start}{solution_end}
The species must be one of: setosa, versicolor, or virginica"""

In [ ]:
# ============================================================================
# STEP 3: Create Chat Template
# ============================================================================
chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{reasoning_start}'")
tokenizer.chat_template = chat_template


In [ ]:
# ============================================================================
# STEP 4: Load and Prepare Iris Dataset
# ============================================================================
from sklearn.datasets import load_iris

# Load iris dataset
iris = load_iris()
iris_df = pd.DataFrame(
    data=iris.data,
    columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
)
iris_df['species'] = iris.target
iris_df['species_name'] = iris_df['species'].map({
    0: 'setosa',
    1: 'versicolor',
    2: 'virginica'
})

print(f"Iris dataset shape: {iris_df.shape}")
print(iris_df.head())

Iris dataset shape: (150, 6)
   sepal_length  sepal_width  petal_length  petal_width  species species_name
0           5.1          3.5           1.4          0.2        0       setosa
1           4.9          3.0           1.4          0.2        0       setosa
2           4.7          3.2           1.3          0.2        0       setosa
3           4.6          3.1           1.5          0.2        0       setosa
4           5.0          3.6           1.4          0.2        0       setosa


In [ ]:
# ============================================================================
# STEP 5: Create Reasoning Examples for Pre-training
# ============================================================================
def create_reasoning_example(row):
    """Create a reasoning example for each flower"""
    species = row['species_name']

    # Create different reasoning styles
    reasoning_templates = [
        f"Looking at the measurements:\n"
        f"- Sepal length: {row['sepal_length']} cm\n"
        f"- Sepal width: {row['sepal_width']} cm\n"
        f"- Petal length: {row['petal_length']} cm\n"
        f"- Petal width: {row['petal_width']} cm\n\n"
        f"The petal length of {row['petal_length']} cm is {'very short' if row['petal_length'] < 2 else 'moderate' if row['petal_length'] < 5 else 'long'}. "
        f"The petal width of {row['petal_width']} cm is {'narrow' if row['petal_width'] < 1 else 'medium' if row['petal_width'] < 1.8 else 'wide'}. "
        f"Based on these characteristics, this appears to be {species}.",

        f"Analyzing the flower:\n"
        f"Sepal: {row['sepal_length']}cm × {row['sepal_width']}cm\n"
        f"Petal: {row['petal_length']}cm × {row['petal_width']}cm\n\n"
        f"{'Setosa typically has very small petals (< 2cm).' if species == 'setosa' else ''}"
        f"{'Versicolor has medium-sized petals (3-5cm).' if species == 'versicolor' else ''}"
        f"{'Virginica has large petals (> 5cm).' if species == 'virginica' else ''}"
        f" This matches {species}.",
    ]

    reasoning = np.random.choice(reasoning_templates)
    return reasoning

# Create pre-training dataset (use 30 samples)
pretrain_df = iris_df.sample(n=30, random_state=42).copy()
pretrain_df['reasoning'] = pretrain_df.apply(create_reasoning_example, axis=1)

def format_pretrain_dataset(row):
    problem = f"""Classify this iris flower:
Sepal Length: {row['sepal_length']} cm
Sepal Width: {row['sepal_width']} cm
Petal Length: {row['petal_length']} cm
Petal Width: {row['petal_width']} cm"""

    final_prompt = (
        reasoning_start + row['reasoning'] + reasoning_end +
        solution_start + row['species_name'] + solution_end
    )

    return {
        "Messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": problem},
            {"role": "assistant", "content": final_prompt},
        ]
    }

pretrain_dataset = pretrain_df.apply(format_pretrain_dataset, axis=1).tolist()
pretrain_dataset = pd.DataFrame(pretrain_dataset)

# Tokenize
pretrain_dataset["text"] = tokenizer.apply_chat_template(
    pretrain_dataset["Messages"].values.tolist(),
    tokenize=False
)
pretrain_dataset = Dataset.from_pandas(pretrain_dataset)

print(f"\nPre-training dataset size: {len(pretrain_dataset)}")
print("\nExample formatted message:")
print(tokenizer.apply_chat_template(pretrain_dataset[0]["Messages"], tokenize=False))



Pre-training dataset size: 30

Example formatted message:
You are a botanist analyzing iris flowers.
Given measurements of an iris flower, analyze the features carefully.
Place your reasoning between <start_working_out> and <end_working_out>.
Then, provide the species name between <SOLUTION></SOLUTION>
The species must be one of: setosa, versicolor, or virginica<|endoftext|>Classify this iris flower:
Sepal Length: 6.1 cm
Sepal Width: 2.8 cm
Petal Length: 4.7 cm
Petal Width: 1.2 cm<start_working_out>Analyzing the flower:
Sepal: 6.1cm × 2.8cm
Petal: 4.7cm × 1.2cm

Versicolor has medium-sized petals (3-5cm). This matches versicolor.<end_working_out><SOLUTION>versicolor</SOLUTION><|endoftext|>


In [ ]:
# ============================================================================
# STEP 6: Pre-Fine-Tune for Format
# ============================================================================
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=pretrain_dataset,
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        warmup_steps=5,
        num_train_epochs=3,
        learning_rate=2e-4,
        logging_steps=5,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

print("\n" + "="*50)
print("Starting pre-training...")
print("="*50)
trainer.train()

# Test the model
text = tokenizer.apply_chat_template(
    pretrain_dataset[0]["Messages"][:2],
    tokenize=False,
    add_generation_prompt=True,
)

from transformers import TextStreamer
print("\n" + "="*50)
print("Testing pre-trained model:")
print("="*50)
_ = model.generate(
    **tokenizer(text, return_tensors="pt").to("cuda"),
    temperature=0,
    max_new_tokens=512,
    streamer=TextStreamer(tokenizer, skip_prompt=False),
)

# Clean up
del pretrain_dataset
torch.cuda.empty_cache()
import gc
gc.collect()


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/30 [00:00<?, ? examples/s]


Starting pre-training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 30 | Num Epochs = 3 | Total steps = 45
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)


Step,Training Loss
5,1.355300
10,0.300100
15,0.080700
20,0.074000
25,0.073400
30,0.047100
35,0.030900
40,0.031600
45,0.033100


Unsloth: Will smartly offload gradients to save VRAM!

Testing pre-trained model:
You are a botanist analyzing iris flowers.
Given measurements of an iris flower, analyze the features carefully.
Place your reasoning between <start_working_out> and <end_working_out>.
Then, provide the species name between <SOLUTION></SOLUTION>
The species must be one of: setosa, versicolor, or virginica<|endoftext|>Classify this iris flower:
Sepal Length: 6.1 cm
Sepal Width: 2.8 cm
Petal Length: 4.7 cm
Petal Width: 1.2 cm<start_working_out>Looking at the measurements:
- Sepal length: 6.1 cm
- Sepal width: 2.8 cm
- Petal length: 4.7 cm
- Petal width: 1.2 cm

The petal length of 4.7 cm is moderate. The petal width of 1.2 cm is medium. Based on these characteristics, this appears to be versicolor.<end_working_out><SOLUTION>versicolor</SOLUTION><|endoftext|>


2322

In [ ]:
# ============================================================================
# STEP 7: Prepare GRPO Dataset
# ============================================================================
# Use remaining samples for GRPO training
grpo_df = iris_df[~iris_df.index.isin(pretrain_df.index)].copy()

def format_grpo_dataset(row):
    problem = f"""Classify this iris flower:
Sepal Length: {row['sepal_length']} cm
Sepal Width: {row['sepal_width']} cm
Petal Length: {row['petal_length']} cm
Petal Width: {row['petal_width']} cm"""

    return {
        "prompt": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": problem},
        ],
        "answer": row['species_name']
    }

grpo_dataset = grpo_df.apply(format_grpo_dataset, axis=1).tolist()
grpo_dataset = pd.DataFrame(grpo_dataset)
grpo_dataset = Dataset.from_pandas(grpo_dataset)

print(f"\nGRPO dataset size: {len(grpo_dataset)}")
print(f"First example: {grpo_dataset[0]}")



GRPO dataset size: 120
First example: {'prompt': [{'content': 'You are a botanist analyzing iris flowers.\nGiven measurements of an iris flower, analyze the features carefully.\nPlace your reasoning between <start_working_out> and <end_working_out>.\nThen, provide the species name between <SOLUTION></SOLUTION>\nThe species must be one of: setosa, versicolor, or virginica', 'role': 'system'}, {'content': 'Classify this iris flower:\nSepal Length: 5.1 cm\nSepal Width: 3.5 cm\nPetal Length: 1.4 cm\nPetal Width: 0.2 cm', 'role': 'user'}], 'answer': 'setosa'}


In [ ]:
# ============================================================================
# STEP 8: Define Reward Functions
# ============================================================================
solution_end_regex = r"</SOLUTION>[\s]{0,}" + \
    "(?:" + re.escape(tokenizer.eos_token) + ")?"

match_format = re.compile(
    rf"{reasoning_end}.*?"
    rf"{solution_start}(.+?){solution_end_regex}"
    rf"[\s]{{0,}}$",
    flags=re.MULTILINE | re.DOTALL
)

def match_format_exactly(completions, **kwargs):
    """Reward correct formatting"""
    scores = []
    for completion in completions:
        response = completion[0]["content"]
        score = 3.0 if match_format.search(response) is not None else 0
        scores.append(score)
    return scores

def match_format_approximately(completions, **kwargs):
    """Reward partial formatting"""
    scores = []
    for completion in completions:
        response = completion[0]["content"]
        score = 0
        score += 0.5 if response.count(reasoning_end) == 1 else -1.0
        score += 0.5 if response.count(solution_start) == 1 else -1.0
        score += 0.5 if response.count(solution_end) == 1 else -1.0
        scores.append(score)
    return scores

def check_answer(prompts, completions, answer, **kwargs):
    """Reward correct species classification"""
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1).strip().lower()
        if (guess := match_format.search(r)) is not None else None
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(-2.0)
            continue

        true_answer = true_answer.strip().lower()

        # Exact match gets high reward
        if guess == true_answer:
            scores.append(5.0)
        # Partial match (contains the right species)
        elif true_answer in guess or guess in true_answer:
            scores.append(3.0)
        else:
            scores.append(-3.0)

    # Removed: scores.append(score)  # This line was wrong
    return scores

global PRINTED_TIMES
PRINTED_TIMES = 0
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5

def print_examples(prompts, completions, answer, **kwargs):
    """Print examples periodically"""
    global PRINTED_TIMES, PRINT_EVERY_STEPS

    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        question = prompts[0][-1]["content"]
        response = completions[0][0]["content"]

        extracted = None
        if (match := match_format.search(response)):
            extracted = match.group(1).strip()

        print("\n" + "="*60)
        print(f"STEP {PRINTED_TIMES}")
        print("="*60)
        print(f"Question:\n{question}")
        print(f"\nTrue Answer: {answer[0]}")
        print(f"\nModel Response:\n{response}")
        print(f"\nExtracted Answer: {extracted}")
        print("="*60)

    PRINTED_TIMES += 1
    return [0] * len(completions)  # Neutral reward


In [ ]:
# ============================================================================
# STEP 9: GRPO Training
# ============================================================================
from vllm import SamplingParams
from trl import GRPOConfig, GRPOTrainer

max_prompt_length = 512
max_completion_length = max_seq_length - max_prompt_length

vllm_sampling_params = SamplingParams(
    min_p=0.1,
    top_p=1.0,
    top_k=-1,
    seed=3407,
    stop=[tokenizer.eos_token],
    include_stop_str_in_output=True,
)

training_args = GRPOConfig(
    vllm_sampling_params=vllm_sampling_params,
    temperature=1.0,
    learning_rate=5e-6,
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    optim="adamw_8bit",
    logging_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_generations=4,
    max_prompt_length=max_prompt_length,
    max_completion_length=max_completion_length,
    num_train_epochs=2,
    save_steps=50,
    output_dir="outputs",
    report_to="none",
)

grpo_trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    args=training_args,  # Changed from 'config' to 'args'
    train_dataset=grpo_dataset,
    reward_funcs=[
        match_format_exactly,
        match_format_approximately,
        check_answer,
        print_examples,
    ],
)

print("\n" + "="*50)
print("Starting GRPO training...")
print("="*50)
grpo_trainer.train()


Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4

Starting GRPO training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 120 | Num Epochs = 2 | Total steps = 120
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)



STEP 0
Question:
Classify this iris flower:
Sepal Length: 5.1 cm
Sepal Width: 3.3 cm
Petal Length: 1.7 cm
Petal Width: 0.5 cm

True Answer: setosa

Model Response:
Looking at the measurements:
- Sepal length: 5.1 cm
- Sepal width: 3.3 cm
- Petal length: 1.7 cm
- Petal width: 0.5 cm

The petal length of 1.7 cm is very short. The petal width of 0.5 cm is narrow. Based on these characteristics, this appears to be setosa.<end_working_out><SOLUTION>setosa</SOLUTION>

Extracted Answer: setosa


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_approximately / mean,rewards / match_format_approximately / std,rewards / check_answer / mean,rewards / check_answer / std,rewards / print_examples / mean,rewards / print_examples / std
1,0.000800,6.500000,2.000000,101.625000,101.000000,102.000000,0.000000,101.625000,101.000000,102.000000,0.416640,3.000000,0.000000,1.500000,0.000000,2.000000,4.140394,0.000000,0.000000
2,0.000800,2.500000,2.000000,100.750000,98.000000,102.000000,0.000000,100.750000,98.000000,102.000000,0.421818,3.000000,0.000000,1.500000,0.000000,-2.000000,2.828427,0.000000,0.000000
3,0.000900,6.500000,2.000000,101.625000,101.000000,102.000000,0.000000,101.625000,101.000000,102.000000,0.426140,3.000000,0.000000,1.500000,0.000000,2.000000,4.140394,0.000000,0.000000
4,0.000800,7.500000,2.309401,100.125000,89.000000,102.000000,0.000000,100.125000,89.000000,102.000000,0.424559,3.000000,0.000000,1.500000,0.000000,3.000000,3.703280,0.000000,0.000000
5,0.001000,4.500000,4.309401,102.500000,101.000000,111.000000,0.000000,102.500000,101.000000,111.000000,0.475411,3.000000,0.000000,1.500000,0.000000,0.000000,4.140394,0.000000,0.000000
6,0.001400,4.500000,2.000000,101.500000,101.000000,103.000000,0.000000,101.500000,101.000000,103.000000,0.696504,3.000000,0.000000,1.500000,0.000000,0.000000,4.140394,0.000000,0.000000
7,0.001500,8.500000,2.000000,101.375000,101.000000,102.000000,0.000000,101.375000,101.000000,102.000000,0.766882,3.000000,0.000000,1.500000,0.000000,4.000000,2.828427,0.000000,0.000000
8,0.000900,7.500000,2.309401,101.750000,101.000000,102.000000,0.000000,101.750000,101.000000,102.000000,0.435775,3.000000,0.000000,1.500000,0.000000,3.000000,3.703280,0.000000,0.000000
9,0.000900,2.500000,2.000000,101.125000,101.000000,102.000000,0.000000,101.125000,101.000000,102.000000,0.439075,3.000000,0.000000,1.500000,0.000000,-2.000000,2.828427,0.000000,0.000000
10,0.001000,7.500000,2.309401,103.375000,101.000000,114.000000,0.000000,103.375000,101.000000,114.000000,0.484617,3.000000,0.000000,1.500000,0.000000,3.000000,3.703280,0.000000,0.000000



STEP 5
Question:
Classify this iris flower:
Sepal Length: 5.5 cm
Sepal Width: 2.4 cm
Petal Length: 3.7 cm
Petal Width: 1.0 cm

True Answer: versicolor

Model Response:
Looking at the measurements:
- Sepal length: 5.5 cm
- Sepal width: 2.4 cm
- Petal length: 3.7 cm
- Petal width: 1.0 cm

The petal length of 3.7 cm is moderate. The petal width of 1.0 cm is medium. Based on these characteristics, this appears to be versicolor.<end_working_out><SOLUTION>versicolor</SOLUTION>

Extracted Answer: versicolor

STEP 10
Question:
Classify this iris flower:
Sepal Length: 6.7 cm
Sepal Width: 3.1 cm
Petal Length: 4.7 cm
Petal Width: 1.5 cm

True Answer: versicolor

Model Response:
Looking at the measurements:
- Sepal length: 6.7 cm
- Sepal width: 3.1 cm
- Petal length: 4.7 cm
- Petal width: 1.5 cm

The petal length of 4.7 cm is moderate. The petal width of 1.5 cm is medium. Based on these characteristics, this appears to be versicolor.<end_working_out><SOLUTION>versicolor</SOLUTION>

Extracted Answ

TrainOutput(global_step=120, training_loss=0.0010456751154075997, metrics={'train_runtime': 1164.2053, 'train_samples_per_second': 0.206, 'train_steps_per_second': 0.103, 'total_flos': 0.0, 'train_loss': 0.0010456751154075997})

In [ ]:
# ============================================================================
# STEP 10: Test the Model
# ============================================================================
print("\n" + "="*50)
print("Testing final model on new examples:")
print("="*50)

test_examples = [
    {"sepal_length": 5.1, "sepal_width": 3.5, "petal_length": 1.4, "petal_width": 0.2},
    {"sepal_length": 6.7, "sepal_width": 3.0, "petal_length": 5.2, "petal_width": 2.3},
    {"sepal_length": 5.9, "sepal_width": 3.0, "petal_length": 4.2, "petal_width": 1.5},
]

for i, example in enumerate(test_examples):
    problem = f"""Classify this iris flower:
Sepal Length: {example['sepal_length']} cm
Sepal Width: {example['sepal_width']} cm
Petal Length: {example['petal_length']} cm
Petal Width: {example['petal_width']} cm"""

    text = tokenizer.apply_chat_template(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": problem},
        ],
        tokenize=False,
        add_generation_prompt=True,
    )

    print(f"\n{'='*60}")
    print(f"Test Example {i+1}")
    print('='*60)
    _ = model.generate(
        **tokenizer(text, return_tensors="pt").to("cuda"),
        temperature=0.3,
        max_new_tokens=512,
        streamer=TextStreamer(tokenizer, skip_prompt=True),
    )


Testing final model on new examples:

Test Example 1
Looking at the measurements:
- Sepal length: 5.1 cm
- Sepal width: 3.5 cm
- Petal length: 1.4 cm
- Petal width: 0.2 cm

The petal length of 1.4 cm is very short. The petal width of 0.2 cm is narrow. Based on these characteristics, this appears to be setosa.<end_working_out><SOLUTION>setosa</SOLUTION><|endoftext|>

Test Example 2
Looking at the measurements:
- Sepal length: 6.7 cm
- Sepal width: 3.0 cm
- Petal length: 5.2 cm
- Petal width: 2.3 cm

The petal length of 5.2 cm is long. The petal width of 2.3 cm is wide. Based on these characteristics, this appears to be virginica.<end_working_out><SOLUTION>virginica</SOLUTION><|endoftext|>

Test Example 3
Looking at the measurements:
- Sepal length: 5.9 cm
- Sepal width: 3.0 cm
- Petal length: 4.2 cm
- Petal width: 1.5 cm

The petal length of 4.2 cm is moderate. The petal width of 1.5 cm is medium. Based on these characteristics, this appears to be versicolor.<end_working_out><SOLUTION>

In [ ]:
# ============================================================================
# STEP 11: Save the Model
# ============================================================================
model.save_pretrained("iris_reasoning_model")
tokenizer.save_pretrained("iris_reasoning_model")
print("\n✓ Model saved to 'iris_reasoning_model'")


✓ Model saved to 'iris_reasoning_model'


In [ ]:
# ============================================================================
# STEP 12: Export to GGUF using Unsloth (Easier Method)
# ============================================================================

print("\n" + "="*50)
print("Exporting to GGUF format using Unsloth...")
print("="*50)

# Quantization options: "f16", "q8_0", "q4_k_m", "q5_k_m"
# f16 = Full precision (largest file, best quality)
# q8_0 = 8-bit quantization (good balance)
# q4_k_m = 4-bit quantization (smallest file, faster)

model.save_pretrained_gguf(
    "iris_reasoning_model",  # Folder name
    tokenizer,
    quantization_method="f16"  # or "q8_0", "q4_k_m", "q5_k_m"
)

print("✓ GGUF model saved!")


Exporting to GGUF format using Unsloth...


Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 8.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 31.17 out of 52.96 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


  0%|          | 0/36 [00:00<?, ?it/s]
We will save to Disk and not RAM now.
100%|██████████| 36/36 [00:24<00:00,  1.46it/s]


Unsloth: Saving tokenizer... Done.
Done.


Unsloth: Converting qwen3 model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at iris_reasoning_model into f16 GGUF format.
The output location will be /content/iris_reasoning_model/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: iris_reasoning_model
INFO:hf-to-gguf:Model architecture: Qwen3ForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-000